# Floating Wave Scenario Analysis

In [1]:
%matplotlib inline

In [2]:
from IPython.display import display, HTML

In [3]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (14.0, 8.0)

In [4]:
import numpy as np
from datetime import datetime

In [5]:
from dtocean_core import start_logging
from dtocean_core.core import Core
from dtocean_core.menu import DataMenu, ModuleMenu, ProjectMenu, ThemeMenu
from dtocean_core.pipeline import Tree, _get_connector
from dtocean_core.strategy import StrategyManager

In [6]:
# Bring up the logger
start_logging()

2016-09-12 12:12:18,140 - INFO - dtocean_core - Begin logging for dtocean_core.


In [7]:
def html_list(x):
    message = "<ul>"
    for name in x:
        message += "<li>{}</li>".format(name)
    message += "</ul>"
    return message
def html_dict(x):
    message = "<ul>"
    for name, status in x.iteritems():
        message += "<li>{}: <b>{}</b></li>".format(name, status)
    message += "</ul>"
    return message
def html_variable(core, project, variable):
    value = variable.get_value(core, project)
    metadata = variable.get_metadata(core)
    name = metadata.title
    units = metadata.units
    message = "<b>{}:</b> {}".format(name, value)
    if units:
        message += " ({})".format(units[0])
    return message

## Create the core, menus and pipeline tree

The core object carrys all the system information and is operated on by the other classes

In [8]:
new_core = Core()
project_menu = ProjectMenu()
module_menu = ModuleMenu()
theme_menu = ThemeMenu()
data_menu = DataMenu()
pipe_tree = Tree()

## Create a new project and tree

In [9]:
project_title = "DTOcean"  
new_project = project_menu.new_project(new_core, project_title)

2016-09-12 12:12:22,822 - INFO - aneris.entity.simulation - Created new Simulation with title "Default"
2016-09-12 12:12:22,822 - INFO - aneris.control.simulation - Datastate with level "initial" stored
2016-09-12 12:12:22,822 - INFO - aneris.control.pipeline - New Hub created for interface ProjectInterface.


## Set the device type

In [10]:
options_branch = pipe_tree.get_branch(new_core, new_project, "System Type Selection")
variable_id = "device.system_type"
my_var = options_branch.get_input_variable(new_core, new_project, variable_id)
my_var.set_raw_interface(new_core, "Wave Floating")
my_var.read(new_core, new_project)

2016-09-12 12:12:22,851 - INFO - aneris.control.data - New "device.system_type" data stored with index JIBNB9
2016-09-12 12:12:22,851 - INFO - aneris.control.simulation - Datastate stored


## Initiate the pipeline
This step will be important when the database is incorporated into the system as it will effect the operation of the pipeline.

In [11]:
project_menu.initiate_pipeline(new_core, new_project)

2016-09-12 12:12:22,868 - INFO - aneris.control.simulation - Datastate with level "system type selection start" stored
2016-09-12 12:12:22,872 - INFO - aneris.control.data - New "hidden.pipeline_active" data stored with index 1U9D0Y
2016-09-12 12:12:22,874 - INFO - aneris.control.simulation - Datastate with level "system type selection output" stored
2016-09-12 12:12:22,875 - INFO - aneris.control.pipeline - New Pipeline created for interface ModuleInterface.
2016-09-12 12:12:22,877 - INFO - aneris.control.pipeline - New Hub created for interface ThemeInterface.


## Discover available modules

In [12]:
names = module_menu.get_available(new_core, new_project)
message = html_list(names)
HTML(message)

## Activate some modules

Note that the order of activation is important and that we can't deactivate yet!

In [13]:
module_menu.activate(new_core, new_project, 'Hydrodynamics')
module_menu.activate(new_core, new_project, 'Electrical Sub Systems')
module_menu.activate(new_core, new_project, 'Mooring and Foundations')

## Activate the Economics and Reliability themes

In [14]:
names = theme_menu.get_available(new_core, new_project)
message = html_list(names)
HTML(message)

In [15]:
theme_menu.activate(new_core, new_project, "Economics")

In [16]:
# Here we are expecting Hydrodynamics
assert _get_connector(new_project, "modules").get_current_interface_name(new_core, new_project) == "Hydrodynamics"

In [17]:
from aneris.utilities.analysis import get_variable_network, count_atomic_variables

In [18]:
req_inputs, opt_inputs, outputs, req_inter, opt_inter = get_variable_network(new_core.control,
                                                                             new_project.get_pool(),
                                                                             new_project.get_simulation(),
                                                                             "modules")

In [19]:
req_inputs[req_inputs.Type=="Shared"].reset_index()

,index,Type,Interface,Identifier
0,0,Shared,Hydrodynamics,bathymetry.layers
1,1,Shared,Hydrodynamics,corridor.landing_point
2,2,Shared,Hydrodynamics,device.coordinate_system
3,7,Shared,Hydrodynamics,device.power_rating
4,8,Shared,Hydrodynamics,device.system_type
5,21,Shared,Electrical Sub Systems,bathymetry.layers
6,22,Shared,Electrical Sub Systems,farm.direction_of_max_surface_current
7,23,Shared,Electrical Sub Systems,farm.max_surface_current_10_year
8,24,Shared,Electrical Sub Systems,farm.wave_direction_100_year
9,25,Shared,Electrical Sub Systems,device.power_rating


In [20]:
shared_req_inputs = req_inputs[req_inputs.Type=="Shared"]
len(shared_req_inputs["Identifier"].unique())

11

In [21]:
count_atomic_variables(shared_req_inputs["Identifier"].unique(),
                       new_core.data_catalog,
                       "labels",
                       ["TableData",
                        "TableDataColumn",
                        "IndexTable",
                        "LineTable",
                        "LineTableColumn",
                        "TimeTable",
                        "TimeTableColumn"])

11

In [22]:
opt_inputs[opt_inputs.Type=="Shared"].reset_index()

,index,Type,Interface,Identifier
0,1,Shared,Hydrodynamics,farm.nogo_areas
1,5,Shared,Electrical Sub Systems,farm.nogo_areas
2,26,Shared,Electrical Sub Systems,device.prescribed_footprint_radius
3,32,Shared,Mooring and Foundations,device.prescribed_footprint_radius


In [23]:
shared_opt_inputs = opt_inputs[opt_inputs.Type=="Shared"]
len(shared_opt_inputs["Identifier"].unique())

2

In [24]:
count_atomic_variables(shared_opt_inputs["Identifier"].unique(),
                       new_core.data_catalog,
                       "labels",
                       ["TableData",
                        "TableDataColumn",
                        "IndexTable",
                        "LineTable",
                        "LineTableColumn",
                        "TimeTable",
                        "TimeTableColumn"])

2

In [25]:
req_inter

,Source,Destination,Identifier
0,Hydrodynamics,Electrical Sub Systems,farm.annual_energy
1,Hydrodynamics,Electrical Sub Systems,farm.layout
2,Hydrodynamics,Electrical Sub Systems,farm.mean_power_hist_per_device
3,Hydrodynamics,Mooring and Foundations,farm.main_direction
4,Hydrodynamics,Mooring and Foundations,farm.layout
5,Electrical Sub Systems,Mooring and Foundations,farm.umbilical_seabed_connection


In [26]:
len(req_inter["Identifier"].unique())

5

In [27]:
count_atomic_variables(req_inter["Identifier"].unique(),
                       new_core.data_catalog,
                       "labels",
                       ["TableData",
                        "TableDataColumn",
                        "IndexTable",
                        "LineTable",
                        "LineTableColumn",
                        "TimeTable",
                        "TimeTableColumn"])

5

In [28]:
opt_inter

,Source,Destination,Identifier
0,Hydrodynamics,Electrical Sub Systems,farm.main_direction
1,Hydrodynamics,Mooring and Foundations,device.external_forces
2,Electrical Sub Systems,Mooring and Foundations,farm.substation_props
3,Electrical Sub Systems,Mooring and Foundations,farm.substation_layout
4,Electrical Sub Systems,Mooring and Foundations,farm.substation_cog
5,Electrical Sub Systems,Mooring and Foundations,farm.substation_foundation_location


In [29]:
len(opt_inter["Identifier"].unique())

6

In [30]:
count_atomic_variables(opt_inter["Identifier"].unique(),
                       new_core.data_catalog,
                       "labels",
                       ["TableData",
                        "TableDataColumn",
                        "IndexTable",
                        "LineTable",
                        "LineTableColumn",
                        "TimeTable",
                        "TimeTableColumn"])

20

In [31]:
hyrdo_req_inputs = req_inputs.loc[req_inputs['Interface'] == 'Hydrodynamics']
len(hyrdo_req_inputs["Identifier"].unique())

21

In [32]:
count_atomic_variables(hyrdo_req_inputs["Identifier"].unique(),
                       new_core.data_catalog,
                       "labels",
                       ["TableData",
                        "TableDataColumn",
                        "IndexTable",
                        "LineTable",
                        "LineTableColumn",
                        "TimeTable",
                        "TimeTableColumn"])

23

In [33]:
hyrdo_opt_inputs = opt_inputs.loc[opt_inputs['Interface'] == 'Hydrodynamics']
len(hyrdo_opt_inputs["Identifier"].unique())

5

In [34]:
count_atomic_variables(hyrdo_opt_inputs["Identifier"].unique(),
                       new_core.data_catalog,
                       "labels",
                       ["TableData",
                        "TableDataColumn",
                        "IndexTable",
                        "LineTable",
                        "LineTableColumn",
                        "TimeTable",
                        "TimeTableColumn"])

5

In [35]:
electro_req_inputs = req_inputs.loc[req_inputs['Interface'] == 'Electrical Sub Systems']
len(electro_req_inputs["Identifier"].unique())

32

In [36]:
count_atomic_variables(electro_req_inputs["Identifier"].unique(),
                       new_core.data_catalog,
                       "labels",
                       ["TableData",
                        "TableDataColumn",
                        "IndexTable",
                        "LineTable",
                        "LineTableColumn",
                        "TimeTable",
                        "TimeTableColumn"])

186

In [37]:
electro_opt_inputs = opt_inputs.loc[opt_inputs['Interface'] == 'Electrical Sub Systems']
len(electro_opt_inputs["Identifier"].unique())

25

In [38]:
count_atomic_variables(electro_opt_inputs["Identifier"].unique(),
                       new_core.data_catalog,
                       "labels",
                       ["TableData",
                        "TableDataColumn",
                        "IndexTable",
                        "LineTable",
                        "LineTableColumn",
                        "TimeTable",
                        "TimeTableColumn"])

37

In [39]:
moorings_req_inputs = req_inputs.loc[req_inputs['Interface'] == 'Mooring and Foundations']
len(moorings_req_inputs["Identifier"].unique())

77

In [40]:
count_atomic_variables(moorings_req_inputs["Identifier"].unique(),
                       new_core.data_catalog,
                       "labels",
                       ["TableData",
                        "TableDataColumn",
                        "IndexTable",
                        "LineTable",
                        "LineTableColumn",
                        "TimeTable",
                        "TimeTableColumn"])

231

In [41]:
moorings_opt_inputs = opt_inputs.loc[opt_inputs['Interface'] == 'Mooring and Foundations']
len(moorings_opt_inputs["Identifier"].unique())

5

In [42]:
count_atomic_variables(moorings_opt_inputs["Identifier"].unique(),
                       new_core.data_catalog,
                       "labels",
                       ["TableData",
                        "TableDataColumn",
                        "IndexTable",
                        "LineTable",
                        "LineTableColumn",
                        "TimeTable",
                        "TimeTableColumn"])

5

In [43]:
total_req_inputs = req_inputs.loc[req_inputs['Interface'] != 'Shared']
len(total_req_inputs["Identifier"].unique())

117

In [44]:
count_atomic_variables(total_req_inputs["Identifier"].unique(),
                       new_core.data_catalog,
                       "labels",
                       ["TableData",
                        "TableDataColumn",
                        "IndexTable",
                        "LineTable",
                        "LineTableColumn",
                        "TimeTable",
                        "TimeTableColumn"])

427